# IE CDT Time Series Analysis and Signal Processing Lab

## Deep Learning for Time Series Forecasting

#### Fernando Moreno-Pino, University of Oxford (fernando.moreno-pino@eng.ox.ac.uk).

### DeepAR Time Series Forecasting (Training and Testing)

This notebook shows how to train and test your DeepAR time series forecasting model. For a full understanding on how the model is implemented, check the following files: `main.py`, `evaluate.py`, `dataloader.py`, and `net.py`.  You may want to update file paths or parameter settings where needed (e.g., dataset locations).

- You should first check the repository’s README file for information on how to download the datasets and install the Conda environment.

- Disclaimer: the DeepAR implementation used in this tutorial is publicly available here: https://github.com/husnejahan/DeepAR-pytorch/tree/master.

-Note: images in this notebook may not be propertly shown in Github, so please download it and run it locally.

### 1. Setup & Imports


We begin by importing libraries and configuring a logger. Please ensure that all dependencies (e.g. `torch`, `numpy`, `matplotlib`, `tqdm`, etc) are installed. See `requirements.txt` for the full list.

In [1]:
#######################################
#    Imports and Environment Settings #
#######################################

import os
import json
import logging
import numpy as np
import matplotlib
import matplotlib.pyplot as plt

# For a notebook, we can display plots inline:
%matplotlib inline

import torch
from torch import nn
import torch.optim as optim
from torch.utils.data import DataLoader, Dataset, Sampler, RandomSampler
from tqdm import tqdm

# Make sure GPU is available if you have one
cuda_exist = torch.cuda.is_available()
device = torch.device("cuda" if cuda_exist else "cpu")

print(f"Using device: {device}")

Using device: cpu


### 2. Utility Functions
The original code uses `utils.py` for parameter loading, checkpoint handling, logging setup, and so forth. Below, we place a subset of those utilities in the notebook for clarity.

Please, note that for this notebook the params are explicitly selected in a cell below, while if you run `main.py`, a file `params.json` contains the default parameters to use during training.

In [2]:
##########################
#    Utility Functions   #
##########################

class Params:
    """
    Loads hyperparameters from a json file if running main.py. In this notebook, the params are selected in a cell below.
    Example structure in params.json:
    {
        "num_class": 370,
        "embedding_dim": 50,
        "cov_dim": 2,
        "lstm_hidden_dim": 50,
        "lstm_layers": 3,
        "lstm_dropout": 0.2,
        "train_window": 24,
        "predict_batch": 64,
        "batch_size": 32,
        "learning_rate": 0.001,
        "num_epochs": 5,
        "predict_steps": 24,
        "predict_start": 100,
        "sample_times": 100
    }
    """
    def __init__(self, json_path):
        with open(json_path) as f:
            params = json.load(f)
            self.__dict__.update(params)
        self.device = device  # inject the device (cuda or cpu)
        self.model_dir = None
        self.plot_dir = None
        self.relative_metrics = False
        self.sampling = False

def set_logger(log_path):
    """
    Sets up the logger to log info in both shell and file `log_path`.
    """
    logger = logging.getLogger()
    logger.setLevel(logging.INFO)
    
    if not logger.handlers:
        # Logging to a file
        file_handler = logging.FileHandler(log_path)
        file_handler.setFormatter(logging.Formatter('%(asctime)s:%(levelname)s: %(message)s'))
        logger.addHandler(file_handler)

        # Logging to console
        stream_handler = logging.StreamHandler()
        stream_handler.setFormatter(logging.Formatter('%(message)s'))
        logger.addHandler(stream_handler)

def save_checkpoint(state, checkpoint, epoch, is_best=False):
    """
    Saves model and training parameters at checkpoint + 'last.pth.tar'
    If is_best==True, also saves checkpoint + 'best.pth.tar'
    """
    filepath = os.path.join(checkpoint, f'epoch_{epoch+1}.pth.tar')
    torch.save(state, filepath)
    last_filepath = os.path.join(checkpoint, 'last.pth.tar')
    torch.save(state, last_filepath)
    if is_best:
        best_filepath = os.path.join(checkpoint, 'best.pth.tar')
        torch.save(state, best_filepath)

def load_checkpoint(checkpoint, model, optimizer=None):
    """
    Loads model parameters (state_dict) from file_path. 
    If optimizer is provided, loads state_dict of optimizer as well.
    """
    if not os.path.exists(checkpoint):
        raise("File doesn't exist {}".format(checkpoint))
    checkpoint_data = torch.load(checkpoint, map_location=device)
    model.load_state_dict(checkpoint_data['state_dict'])
    if optimizer:
        optimizer.load_state_dict(checkpoint_data['optim_dict'])
    return checkpoint_data

def save_dict_to_json(d, json_path):
    """
    Saves dict of floats in json file
    """
    with open(json_path, 'w') as f:
        d = {k: float(v) for k, v in d.items()}
        json.dump(d, f, indent=4)

def plot_all_epoch(data, name, location):
    """
    Plots a list or array of values (e.g. ND or loss) over epochs.
    """
    plt.figure()
    plt.plot(data, label=name)
    plt.title(name)
    plt.legend()
    plt.savefig(os.path.join(location, f'{name}.png'))
    plt.close()

### 3. Data Loading
We implement here the dataloaders (`TrainDataset` and `TestDataset` classes) and `WeightedSampler` class. See Section 7 for information regarding the dataset we will be using.

If the training is excessively slow, you can choose the number of samples to use when calling the sampler `sampler = WeightedSampler(data_dir, args.dataset, max_samples=5000)` (see below). Please, bear in mind that doing this the model won't converge to a proper solution and results reported in test will not be good. Also, if `len(train_loader) < 1000`, plots of the prediction won't be shown (you can easily modify the code to change it).

In [3]:
from __future__ import division
import numpy as np
import torch
import os
import logging
from torch.utils.data import DataLoader, Dataset, Sampler

logger = logging.getLogger('DeepAR.Data')


class TrainDataset(Dataset):
    def __init__(self, data_path, data_name, num_class):
        self.data = np.load(os.path.join(data_path, f'train_data_{data_name}.npy'))
        self.label = np.load(os.path.join(data_path, f'train_label_{data_name}.npy'))
        self.train_len = self.data.shape[0]
        logger.info(f'train_len: {self.train_len}')
        logger.info(f'building datasets from {data_path}...')

    def __len__(self):
        return self.train_len

    def __getitem__(self, index):
        return (self.data[index,:,:-1],int(self.data[index,0,-1]), self.label[index])

class TestDataset(Dataset):
    def __init__(self, data_path, data_name, num_class):
        self.data = np.load(os.path.join(data_path, f'test_data_{data_name}.npy'))
        self.v = np.load(os.path.join(data_path, f'test_v_{data_name}.npy'))
        self.label = np.load(os.path.join(data_path, f'test_label_{data_name}.npy'))
        self.test_len = self.data.shape[0]
        logger.info(f'test_len: {self.test_len}')
        logger.info(f'building datasets from {data_path}...')

    def __len__(self):
        return self.test_len

    def __getitem__(self, index):
        return (self.data[index,:,:-1],int(self.data[index,0,-1]),self.v[index],self.label[index])



class WeightedSampler(Sampler):
    def __init__(self, data_path, data_name, replacement=True, max_samples=None):
        # Load v as before
        v = np.load(os.path.join(data_path, f'train_v_{data_name}.npy'))

        self.weights = torch.as_tensor(
            np.abs(v[:, 0]) / np.sum(np.abs(v[:, 0])),
            dtype=torch.double
        )
        logger.info(f'weights: {self.weights}')

        # If max_samples is specified, reduce the total number of samples
        if max_samples is not None:
            self.num_samples = min(len(self.weights), max_samples)
        else:
            self.num_samples = len(self.weights)

        self.replacement = replacement
        logger.info(f'num samples (per epoch): {self.num_samples}')

    def __iter__(self):
        # Grab a subset of indices based on self.num_samples
        # when replacement=True, use multinomial; otherwise a variation:
        return iter(
            torch.multinomial(self.weights, self.num_samples, self.replacement)
            .tolist()
        )

    def __len__(self):
        return self.num_samples

### 4. DeepAR

In the following cell we implement the `Net` class (an embedding plus the LSTM), along with the Gaussian log-likelihood loss function and the error metrics.

The structure of DeepAR is shown in the following figure (**Left** shows how the model operates during training. **Right** shows how the model operates during forecasting/testing).


![title](notebook_images/model.png)

Usual metrics for forecasting evaluation are the Normalized Deviation (ND) and Root Mean Square Error (RMSE):

\begin{equation} 
	\begin{split}
	\mathrm{ND} &=\frac{\sum_{i, t}\left|z_{i, t}-\hat{z}_{i, t}\right|}{\sum_{i, t}\left|z_{i, t}\right|} \\[5pt]
	\text { RMSE } &=\frac{\sqrt{\frac{1}{N\left(T-t_{0}\right)} \sum_{i, t}\left(z_{i, t}-\hat{z}_{i, t}\right)^{2}}}{\frac{1}{N\left(T-t_{0}\right)} \sum_{i, t}\left|z_{i, t}\right|}\\[5pt]
	\end{split}
\end{equation}

Also, the quantile loss, $QL_{\rho}$, with $\rho \in (0, 1)$ are commonly used:

\begin{equation}
	\begin{split}
	\mathrm{QL}_{\rho}(z, \hat{z})&=2 \frac{\sum_{i, t} P_{\rho}\left(z_{t}^{(i)}, \hat{z}_{t}^{(i)}\right)}{\sum_{i, t}\left|z_{t}^{(i)}\right|},\\
	\quad P_{\rho}(z, \hat{z})&=\left\{\begin{array}{ll}
	\rho(z-\hat{z}) & \text { if } z>\hat{z}, \\
	(1-\rho)(\hat{z}-z) & \text { otherwise }
	\end{array}\right.
	\end{split}
\end{equation}


In [11]:
from IPython.display import Image
PATH = 'notebook_images/model.png'
Image(filename = PATH)

In [4]:
#########################
#    Model Architecture #
#########################

class Net(nn.Module):
    """
    DeepAR model with:
      - an Embedding for categorical features (time series ID)
      - LSTM for sequence processing
      - output layers to predict mean (mu) and std dev (sigma)
    """
    def __init__(self, params):
        super(Net, self).__init__()
        self.params = params
        self.embedding = nn.Embedding(params.num_class, params.embedding_dim)

        self.lstm = nn.LSTM(
            input_size=1 + params.cov_dim + params.embedding_dim,
            hidden_size=params.lstm_hidden_dim,
            num_layers=params.lstm_layers,
            bias=True,
            batch_first=False,
            dropout=params.lstm_dropout
        )
        # Initialise LSTM forget gate bias
        for names in self.lstm._all_weights:
            for name in filter(lambda n: "bias" in n, names):
                bias = getattr(self.lstm, name)
                n = bias.size(0)
                start, end = n // 4, n // 2
                bias.data[start:end].fill_(1.)

        self.relu = nn.ReLU()
        self.distribution_mu = nn.Linear(params.lstm_hidden_dim * params.lstm_layers, 1)
        self.distribution_presigma = nn.Linear(params.lstm_hidden_dim * params.lstm_layers, 1)
        self.distribution_sigma = nn.Softplus()

    def forward(self, x, idx, hidden, cell):
        """
        x: [1, batch_size, 1+cov_dim] (z_{t-1} + x_t)
        idx: [1, batch_size] (time series IDs)
        hidden, cell: LSTM hidden state
        Returns: mu, sigma, hidden, cell
        """
        # Embedding for the time series ID
        onehot_embed = self.embedding(idx)
        lstm_input = torch.cat((x, onehot_embed), dim=2)
        output, (hidden, cell) = self.lstm(lstm_input, (hidden, cell))

        hidden_permute = hidden.permute(1, 2, 0).contiguous().view(hidden.shape[1], -1)
        pre_sigma = self.distribution_presigma(hidden_permute)
        mu = self.distribution_mu(hidden_permute)
        sigma = self.distribution_sigma(pre_sigma)
        return torch.squeeze(mu), torch.squeeze(sigma), hidden, cell

    def init_hidden(self, batch_size):
        return torch.zeros(
            self.params.lstm_layers, batch_size, self.params.lstm_hidden_dim, 
            device=self.params.device
        )

    def init_cell(self, batch_size):
        return torch.zeros(
            self.params.lstm_layers, batch_size, self.params.lstm_hidden_dim, 
            device=self.params.device
        )

    def test(self, x, v_batch, id_batch, hidden, cell, sampling=False):
        """
        Inference method for test time, optionally sampling multiple draws.
        """
        batch_size = x.shape[1]
        if sampling:
            samples = torch.zeros(self.params.sample_times, batch_size, self.params.predict_steps,
                                  device=self.params.device)
            decoder_hidden = hidden
            decoder_cell = cell

            for j in range(self.params.sample_times):
                decoder_hidden = hidden
                decoder_cell = cell
                # copy x so each sample's trajectory is independent
                x_sample = x.clone()  
                for t in range(self.params.predict_steps):
                    mu_de, sigma_de, decoder_hidden, decoder_cell = self(
                        x_sample[self.params.test_predict_start + t].unsqueeze(0),
                        id_batch,
                        decoder_hidden,
                        decoder_cell
                    )
                    gaussian = torch.distributions.normal.Normal(mu_de, sigma_de)
                    pred = gaussian.sample()
                    samples[j, :, t] = pred * v_batch[:, 0] + v_batch[:, 1]
                    if t < (self.params.predict_steps - 1):
                        x_sample[self.params.test_predict_start + t + 1, :, 0] = pred
            sample_mu = torch.median(samples, dim=0)[0]
            sample_sigma = samples.std(dim=0)
            return samples, sample_mu, sample_sigma
        else:
            decoder_hidden = hidden
            decoder_cell = cell
            sample_mu = torch.zeros(batch_size, self.params.predict_steps, device=self.params.device)
            sample_sigma = torch.zeros(batch_size, self.params.predict_steps, device=self.params.device)
            x_sample = x.clone()
            for t in range(self.params.predict_steps):
                mu_de, sigma_de, decoder_hidden, decoder_cell = self(
                    x_sample[self.params.test_predict_start + t].unsqueeze(0),
                    id_batch,
                    decoder_hidden,
                    decoder_cell
                )
                sample_mu[:, t] = mu_de * v_batch[:, 0] + v_batch[:, 1]
                sample_sigma[:, t] = sigma_de * v_batch[:, 0]
                if t < (self.params.predict_steps - 1):
                    x_sample[self.params.test_predict_start + t + 1, :, 0] = mu_de
            return sample_mu, sample_sigma

# The loss function for training
def loss_fn(mu, sigma, labels):
    zero_index = (labels != 0)
    distribution = torch.distributions.normal.Normal(mu[zero_index], sigma[zero_index])
    likelihood = distribution.log_prob(labels[zero_index])
    return -torch.mean(likelihood)


############################
#    Evaluation Metrics    #
############################

def accuracy_ND(mu, labels, relative=False):
    """
    ND = sum(|mu - labels|)/sum(|labels|)
    If relative = True, we do a simple average over the number of samples instead.
    """
    zero_index = (labels != 0)
    if relative:
        diff = torch.mean(torch.abs(mu[zero_index] - labels[zero_index])).item()
        return [diff, 1]
    else:
        diff = torch.sum(torch.abs(mu[zero_index] - labels[zero_index])).item()
        summation = torch.sum(torch.abs(labels[zero_index])).item()
        return [diff, summation]

def accuracy_RMSE(mu, labels, relative=False):
    zero_index = (labels != 0)
    diff = torch.sum((mu[zero_index] - labels[zero_index])**2).item()
    if relative:
        return [diff, torch.sum(zero_index).item(), torch.sum(zero_index).item()]
    else:
        summation = torch.sum(torch.abs(labels[zero_index])).item()
        return [diff, summation, torch.sum(zero_index).item()]

As can been seen in the previous cell, DeepAR parametrizes the Gaussian likelihood using its mean and standard deviation, $\theta = (\mu, \sigma)$, where the mean is given by a function of the network output, and the standard deviation is obtained by applying a softplus activation in order to ensure $\sigma > 0$:



![title](notebook_images/parametrization.png)


### 6. Bringing It All Together
Now we construct the necessary objects, load datasets, and run the training and evaluation loop. This mirrors the code under the `if __name__ == "__main__"` block in `main.py`.

For training the model's parameters, we will maximize the log-likelihood:

![title](notebook_images/loss.png)


In [5]:
###################################
#    Training Code                #
###################################

def train(model, optimizer, loss_fn, train_loader, test_loader, params, epoch):
    """
    Train the model for one epoch on the training set.
    """
    model.train()
    loss_epoch = np.zeros(len(train_loader))

    for i, (train_batch, idx, labels_batch) in enumerate(tqdm(train_loader)):
        optimizer.zero_grad()
        batch_size = train_batch.shape[0]

        train_batch = train_batch.permute(1, 0, 2).to(torch.float32).to(params.device)
        labels_batch = labels_batch.permute(1, 0).to(torch.float32).to(params.device)
        idx = idx.unsqueeze(0).to(params.device)

        loss_val = torch.zeros(1, device=params.device)
        hidden = model.init_hidden(batch_size)
        cell = model.init_cell(batch_size)

        for t in range(params.train_window):
            # If z_t is missing, replace it with mu from last time step
            zero_index = (train_batch[t, :, 0] == 0)
            if t > 0 and torch.sum(zero_index) > 0:
                train_batch[t, zero_index, 0] = mu[zero_index]
            mu, sigma, hidden, cell = model(train_batch[t].unsqueeze(0).clone(), idx, hidden, cell)
            loss_val += loss_fn(mu, sigma, labels_batch[t])

        loss_val.backward()
        optimizer.step()

        loss_val_item = loss_val.item() / params.train_window
        loss_epoch[i] = loss_val_item

        # Optional mid-epoch evaluation on the test set:
        if i % 1000 == 0:
            test_metrics = evaluate(model, loss_fn, test_loader, params, epoch, sample=params.sampling)
            model.train()
            logging.info(f'Iteration {i} | train_loss: {loss_val_item:.4f}')

    return loss_epoch

In [6]:
###################################
#    Evaluation Code              #
###################################

def evaluate(model, loss_fn, test_loader, params, epoch, sample=False):
    """
    Evaluate the model on the test set. Returns a dictionary of metrics.
    """
    model.eval()
    with torch.no_grad():
        total_loss = 0.0
        total_nd_num = 0.0
        total_nd_den = 0.0
        total_rmse_num = 0.0
        total_rmse_den = 0.0
        total_count = 0

        for i, (test_batch, idx, v_batch, labels_batch) in enumerate(test_loader):
            batch_size = test_batch.shape[0]

            test_batch = test_batch.permute(1, 0, 2).to(torch.float32).to(params.device)
            labels_batch = labels_batch.permute(1, 0).to(torch.float32).to(params.device)
            idx = idx.unsqueeze(0).to(params.device)
            v_batch = v_batch.to(torch.float32).to(params.device)

            hidden = model.init_hidden(batch_size)
            cell = model.init_cell(batch_size)

            # Encode the first part of the sequence
            for t in range(params.test_predict_start):
                zero_index = (test_batch[t, :, 0] == 0)
                if t > 0 and torch.sum(zero_index) > 0:
                    test_batch[t, zero_index, 0] = mu[zero_index]
                mu, sigma, hidden, cell = model(test_batch[t].unsqueeze(0).clone(), idx, hidden, cell)

            # Generate predictions
            if sample:
                samples, sample_mu, sample_sigma = model.test(test_batch, v_batch, idx, hidden, cell, sampling=True)
                # For metrics, we can use sample_mu as our point forecast
                mu_out = sample_mu
            else:
                sample_mu, sample_sigma = model.test(test_batch, v_batch, idx, hidden, cell, sampling=False)
                mu_out = sample_mu

            # We only evaluate on the predicted part
            labels_eval = labels_batch[params.test_predict_start: params.test_predict_start + params.predict_steps, :]
            mu_eval = mu_out.permute(1, 0)

            # Expand or contract if needed
            if mu_eval.shape[0] != labels_eval.shape[0]:
                min_len = min(mu_eval.shape[0], labels_eval.shape[0])
                mu_eval = mu_eval[:min_len, :]
                labels_eval = labels_eval[:min_len, :]

            # Flatten or keep as is for ND and RMSE
            nd_val = accuracy_ND(mu_eval, labels_eval, relative=params.relative_metrics)
            rmse_val = accuracy_RMSE(mu_eval, labels_eval, relative=params.relative_metrics)

            total_nd_num += nd_val[0]
            total_nd_den += nd_val[1]
            total_rmse_num += rmse_val[0]
            total_rmse_den += rmse_val[1]
            total_count += rmse_val[2]

        ND = total_nd_num / total_nd_den if total_nd_den != 0 else 0
        RMSE = np.sqrt(total_rmse_num / total_count) if total_count != 0 else 0

    metrics = {'ND': ND, 'RMSE': RMSE}
    logging.info(f"Epoch {epoch+1} Evaluation - ND: {ND:.4f}, RMSE: {RMSE:.4f}")
    return metrics

In [7]:
def train_and_evaluate(model, train_loader, test_loader, optimizer, loss_fn, params, restore_file=None):
    """
    Full training loop, includes checkpointing, best-model tracking, and plotting.
    """
    if restore_file is not None:
        restore_path = os.path.join(params.model_dir, restore_file + '.pth.tar')
        logging.info(f'Restoring parameters from {restore_path}')
        load_checkpoint(restore_path, model, optimizer)

    logging.info('Begin training and evaluation')
    best_test_ND = float('inf')
    train_len = len(train_loader)
    ND_summary = np.zeros(params.num_epochs)
    loss_summary = np.zeros((train_len * params.num_epochs))

    for epoch in range(params.num_epochs):
        logging.info(f'Epoch {epoch+1}/{params.num_epochs}')
        epoch_loss = train(model, optimizer, loss_fn, train_loader, test_loader, params, epoch)
        loss_summary[epoch * train_len : (epoch + 1) * train_len] = epoch_loss

        test_metrics = evaluate(model, loss_fn, test_loader, params, epoch, sample=params.sampling)
        ND_summary[epoch] = test_metrics['ND']
        is_best = (ND_summary[epoch] <= best_test_ND)

        # Save model
        save_checkpoint({
            'epoch': epoch + 1,
            'state_dict': model.state_dict(),
            'optim_dict': optimizer.state_dict()
        }, checkpoint=params.model_dir, epoch=epoch, is_best=is_best)

        if is_best:
            logging.info('- Found new best ND')
            best_test_ND = ND_summary[epoch]
            best_json_path = os.path.join(params.model_dir, 'metrics_test_best_weights.json')
            save_dict_to_json(test_metrics, best_json_path)

        logging.info('Current Best ND is: {:.5f}'.format(best_test_ND))

        # Plots
        plot_all_epoch(ND_summary[:epoch + 1], f"{args_dataset}_ND", params.plot_dir)
        plot_all_epoch(loss_summary[: (epoch + 1)*train_len], f"{args_dataset}_loss", params.plot_dir)

        last_json_path = os.path.join(params.model_dir, 'metrics_test_last_weights.json')
        save_dict_to_json(test_metrics, last_json_path)

Before training the code, we define here the value of **DeepAR's parameters**.

In [8]:
class Params:
    """Simple class for holding hyperparameters."""
    def __init__(self):
        # define them directly in code
        self.learning_rate = 1e-3
        self.batch_size = 64
        self.lstm_layers = 3
        self.num_epochs = 10
        self.train_window = 192     #24 datapoints per day * 7 days
        self.test_window = 192
        self.test_predict_start = 168 
        self.predict_steps = 24
        self.num_class = 370
        self.cov_dim = 4
        self.lstm_hidden_dim = 40
        self.embedding_dim = 20
        self.sample_times = 200
        self.lstm_dropout = 0.1
        self.predict_batch = 256

        # device: CPU or GPU
        self.device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

### 7. Training the model

We will be training the model on the electricity dataset (https://archive.ics.uci.edu/dataset/321/electricityloaddiagrams20112014#, see the Readme file for downloading instructions). It consists os 370 time series with hourly granularity. We will be using time series of length 192 (7 days). During training, we will use 168 steps as conditioning range and 24 as forecasting range.


Please, note that the code (publicly available at https://github.com/husnejahan/DeepAR-pytorch/tree/master) does not use a validation dataset. This implies that they are validating the performance on the test set (which is a very bad practice). The easiest fix would be to obtain a val_set from train_set and a val_loader from it, using that data partition when evaluating the model inside the training loop and using test_loader exclusively at the end of the training to evaluate the final model's performance.



In [9]:
# Replace these with your own paths/names if required
args_dataset = "elect"
data_folder = "data"
model_name = "experiments/base_model"
restore_file = None  # e.g. "epoch_5" if you want to resume
relative_metrics = False
sampling = False

# 1) Load parameters from JSON
model_dir = os.path.join(model_name)
json_path = os.path.join(model_dir, 'params.json')
params = Params()

params.relative_metrics = relative_metrics
params.sampling = sampling
params.model_dir = model_dir
params.plot_dir = os.path.join(model_dir, 'figures')

# Create plot_dir if it doesn't exist
os.makedirs(params.plot_dir, exist_ok=True)

# 2) Prepare logging
set_logger(os.path.join(model_dir, 'train.log'))

# 3) Create Datasets and Loaders
logging.info('Loading datasets...')
data_dir = os.path.join(data_folder, args_dataset)

train_set = TrainDataset(data_dir, args_dataset, params.num_class)
test_set = TestDataset(data_dir, args_dataset, params.num_class)
sampler = WeightedSampler(data_dir, args_dataset)
#sampler = WeightedSampler(data_dir, args_dataset, max_samples=5000)
train_loader = DataLoader(train_set, batch_size=params.batch_size, sampler=sampler, num_workers=0)
test_loader = DataLoader(test_set, batch_size=params.predict_batch, sampler=RandomSampler(test_set), num_workers=0)
logging.info('Loading complete.')

# 4) Instantiate Model
model = Net(params).to(params.device)

logging.info(f'Model:\n{model}')

# 5) Optimiser
optimizer = optim.Adam(model.parameters(), lr=params.learning_rate)

# 6) Loss Function
deepAR_loss = loss_fn

# 7) Train
logging.info(f'Starting training for {params.num_epochs} epoch(s)...')
train_and_evaluate(model, train_loader, test_loader, optimizer, deepAR_loss, params, restore_file=restore_file)
logging.info('Training complete.')

Loading datasets...
train_len: 389101
building datasets from data/elect...
test_len: 2590
building datasets from data/elect...
weights: tensor([4.5579e-08, 4.7663e-08, 4.8446e-08,  ..., 8.4827e-05, 8.4776e-05,
        8.4272e-05], dtype=torch.float64)
num samples (per epoch): 5000
Loading complete.
Model:
Net(
  (embedding): Embedding(370, 20)
  (lstm): LSTM(25, 40, num_layers=3, dropout=0.1)
  (relu): ReLU()
  (distribution_mu): Linear(in_features=120, out_features=1, bias=True)
  (distribution_presigma): Linear(in_features=120, out_features=1, bias=True)
  (distribution_sigma): Softplus(beta=1.0, threshold=20.0)
)
Starting training for 10 epoch(s)...
Begin training and evaluation
Epoch 1/10
  0%|          | 0/79 [00:00<?, ?it/s]Epoch 1 Evaluation - ND: 0.8052, RMSE: 17950.9151
Iteration 0 | train_loss: 1.4016
100%|██████████| 79/79 [00:15<00:00,  5.12it/s]
Epoch 1 Evaluation - ND: 0.2715, RMSE: 8077.7871
- Found new best ND
Current Best ND is: 0.27146
Epoch 2/10
  0%|          | 0/79

At the beginning of training (when finishing the first epoch), the predictions of the model will look something like this:

![title](notebook_images/0.png)

At epoch 11:

![title](notebook_images/11.png)

With loss function (all iterations):

![title](notebook_images/elect_loss_summary.png)

And ND error (per epoch):

![title](notebook_images/elect_ND_summary.png)

To achieve performance comparable to the values reported in the paper, you should train for at least 20 epochs.